In [17]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from analysis_functions import Analysis
import sys
sys.path.append("../")
from vqc.vqc_circuits import UQC

First thing I need to do is to be able to create the circuit with the correct weights in pennylane

In [18]:
uqc_2qubits_path = "../../results/uqc_entanglement/2_qubit_uqc_cz"
uqc_4qubits_path = "../../results/uqc_entanglement/4_qubit_uqc_cz"

uqc_2qubits = Analysis(uqc_2qubits_path)
uqc_4qubits = Analysis(uqc_4qubits_path)

Let's try to use negativity to measure the entanglement of the variational quantum circuits

In [40]:
num_qubits = 2
num_layers = 5
entangling_type = "CZ"
vqc = UQC(num_qubits, num_layers, entangling_type)

In [41]:
vqc.circuit

(0, 0): ───Rz(x0_0)───Ry(theta0)───@───Rz(x1_0)───Ry(theta2)───@───Rz(x2_0)───Ry(theta4)───@───Rz(x3_0)───Ry(theta6)───@───Rz(x4_0)───Ry(theta8)───
                                   │                           │                           │                           │
(0, 1): ───Rz(x0_1)───Ry(theta1)───@───Rz(x1_1)───Ry(theta3)───@───Rz(x2_1)───Ry(theta5)───@───Rz(x3_1)───Ry(theta7)───@───Rz(x4_1)───Ry(theta9)───

In [42]:
uqc_weights = uqc_2qubits.get_final_weights()
uqc_weights[0]

[array([[0.22377096, 2.4907498 , 0.7647595 , 0.11703885, 2.49186   ,
         0.5816616 , 2.6380851 , 0.5808931 , 0.5977255 , 0.10605836]],
       dtype=float32),
 array([[[-2.3252773 , -0.45936522, -0.05628812,  0.7296369 ],
         [ 0.17676501,  0.06906239,  0.43049112,  1.820161  ]],
 
        [[-0.7617193 , -0.52727914, -0.3803747 , -0.19308297],
         [ 0.01132563,  0.26265895, -0.96870285, -0.8290429 ]],
 
        [[-0.16651799,  0.11960864,  1.7842529 , -0.04611189],
         [ 0.24644308, -0.7475598 , -1.0119281 , -0.12901615]],
 
        [[-0.1548171 ,  0.08413836,  1.8281894 ,  0.6898526 ],
         [-0.6059063 ,  0.6271617 , -1.2207141 ,  0.5451319 ]],
 
        [[-0.10349423,  0.00413808,  1.0261043 , -0.00749924],
         [-0.67405254,  0.7457811 , -0.15448521,  1.2762944 ]]],
       dtype=float32),
 array([[-4.3867836e-03, -2.4723676e-01],
        [-8.9729235e-02,  1.7556483e-01],
        [ 1.7635074e-01, -3.1621188e-01],
        [ 2.3235589e-04,  4.6824500e-01],
  

We dont need the output scaling weights, so lets start by removing them

In [43]:
uqc_weights = uqc_weights[0][:-1]

Let's put the weights in the correct shapes

In [44]:
uqc_weights[0]

array([[0.22377096, 2.4907498 , 0.7647595 , 0.11703885, 2.49186   ,
        0.5816616 , 2.6380851 , 0.5808931 , 0.5977255 , 0.10605836]],
      dtype=float32)

In [45]:
uqc_weights[0] = uqc_weights[0].reshape(num_layers, num_qubits)
uqc_weights

[array([[0.22377096, 2.4907498 ],
        [0.7647595 , 0.11703885],
        [2.49186   , 0.5816616 ],
        [2.6380851 , 0.5808931 ],
        [0.5977255 , 0.10605836]], dtype=float32),
 array([[[-2.3252773 , -0.45936522, -0.05628812,  0.7296369 ],
         [ 0.17676501,  0.06906239,  0.43049112,  1.820161  ]],
 
        [[-0.7617193 , -0.52727914, -0.3803747 , -0.19308297],
         [ 0.01132563,  0.26265895, -0.96870285, -0.8290429 ]],
 
        [[-0.16651799,  0.11960864,  1.7842529 , -0.04611189],
         [ 0.24644308, -0.7475598 , -1.0119281 , -0.12901615]],
 
        [[-0.1548171 ,  0.08413836,  1.8281894 ,  0.6898526 ],
         [-0.6059063 ,  0.6271617 , -1.2207141 ,  0.5451319 ]],
 
        [[-0.10349423,  0.00413808,  1.0261043 , -0.00749924],
         [-0.67405254,  0.7457811 , -0.15448521,  1.2762944 ]]],
       dtype=float32),
 array([[-4.3867836e-03, -2.4723676e-01],
        [-8.9729235e-02,  1.7556483e-01],
        [ 1.7635074e-01, -3.1621188e-01],
        [ 2.3235589e

In [51]:
def uqc_layer(wires, data, rotational_weights, input_weights, bias_weights):
        [qml.RZ(np.dot(2 * input_weights[i], data) + bias_weights[i] , wires[i]) for i in range(len(wires))]
        [qml.RY(2 * rotational_weights[i], wires[i]) for i in range(len(wires))]

def uqc(params, num_qubits, num_layers, data):
        rotational_weights = params[0]
        input_weights = params[1]
        bias_weights = params[2]
        for l in range(num_layers):
            uqc_layer(range(num_qubits), data, rotational_weights[l], input_weights[l], bias_weights[l])
        if num_qubits == 2:
            return qml.expval(qml.PauliZ(0))
        elif num_qubits == 4:
            return qml.expval(qml.PauliZ(0))
        
dev = qml.device("default.qubit", wires = num_qubits)

circuit = qml.QNode(uqc, dev)

circuit(uqc_weights, num_qubits, num_layers, 2)

AttributeError: 'QNode' object has no attribute 'SVGCircuit'